# GOALS

* Decide on viewcount threshold to eliminate views?
* Get feature columns working
* Build an LDA model

In [1]:
## testing printing output from console
import subprocess
cmd = [ 'echo', '"Welcome to my PySpark analysis of some StackExchange Data"' ]
output = subprocess.Popen( cmd, stdout=subprocess.PIPE ).communicate()[0]
print(output)

b'"Welcome to my PySpark analysis of some StackExchange Data"\n'


In [2]:
import numpy as np
import pandas as pd
from datetime import datetime

# Load PySpark and Data Structures

In [3]:
%run -i '1-load-pyspark-and-structs.py'

The Spark UI is available at: http://192.168.0.26:4040/ and the defaultParallelism is 4
The data_array is: ['english', 'math', 'rus_stackoverflow', 'stackoverflow', 'superuser']


# Load Data

In [4]:
%%time
print(datetime.now().time())
#%run -i '2-load-datasets.py'

14:43:14.065920
CPU times: user 279 µs, sys: 185 µs, total: 464 µs
Wall time: 363 µs


In [5]:
def show_spark_df(df, n=5):
    '''
    function to better print spark df entries
    '''
    display(pd.DataFrame(df.head(n), columns=df.columns))

# Clean Data

In [6]:
%%time
print(datetime.now().time())
#%run -i '3-clean-datasets.py'

14:43:14.096546
CPU times: user 202 µs, sys: 65 µs, total: 267 µs
Wall time: 228 µs


# EDA (optional)

In [7]:
%%time
print(datetime.now().time())
#%run -i '4-eda.py'

#NB TO DO: Find threshold to delete low views to make sure users that can vote have seen the question

'''
vc_thresh_data = {}

## finding means of viewcounts across fora

for i in data_array:
    vc_thresh_data[i] = datasets[i].select("viewcount").rdd.flatMap(lambda x: x).mean()

vc_thresh_data'''

14:43:14.103846
CPU times: user 350 µs, sys: 163 µs, total: 513 µs
Wall time: 409 µs


# Define Ravi Target Variable and Export Clean Data

In [8]:
%%time
print(datetime.now().time())
#%run -i '5-define-target.py'

14:43:14.112001
CPU times: user 145 µs, sys: 60 µs, total: 205 µs
Wall time: 194 µs


In [9]:
%%time

## load clean data
for i in data_array:
    datasets[i] = (
        spark
        .read
        .load(f'clean-data/{i}.parquet')
)

CPU times: user 3.68 ms, sys: 2.28 ms, total: 5.96 ms
Wall time: 3.99 s


# Create Results Dictionary

In [10]:
RESULTS = {}

# Silly Mean and Median Models

In [11]:
from pyspark.sql.functions import array, lit, struct

## create median and mean dictionaries
y_ravi_medians = {}
y_ravi_means = {}

## create constant median and mean columns for evaluation baseline
for i in data_array:
    y_ravi_medians[i] = datasets[i].approxQuantile('y_ravi', [0.5], 0.25)[0]
    y_ravi_means[i] = datasets[i].select('y_ravi').rdd.flatMap(lambda x: x).mean()

In [12]:
## import rmse evaluator
from pyspark.ml.evaluation import RegressionEvaluator

## create baselines dictionary
baselines = {}

for i in data_array:

    ## train silly median model
    datasets[i] = datasets[i].withColumn('median_pred', lit(y_ravi_medians[i]))

    ## evaluate silly median model
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="y_ravi", predictionCol="median_pred")
    baselines[i] = evaluator.evaluate(datasets[i])

    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m median\033[0m model is {baselines[i]}")

    ## store as dictionary inside RESULTS dictionary, initiating dataset name entries first
    RESULTS[i] = {'silly_median': [baselines[i], 0]}

The root-mean-square error of english's median model is 0.011407298175493769
The root-mean-square error of math's median model is 0.01948979798344224
The root-mean-square error of rus_stackoverflow's median model is 0.018921842554517707
The root-mean-square error of stackoverflow's median model is 0.014281609584952339
The root-mean-square error of superuser's median model is 0.010593612389712697


In [13]:
for i in data_array:
    
    ## train silly mean model
    datasets[i] = datasets[i].withColumn('mean_pred', lit(y_ravi_means[i]))

    ## evaluate silly mean model
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="y_ravi", predictionCol="mean_pred")
    rmse = evaluator.evaluate(datasets[i])

    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m mean\033[0m model is {rmse}")

    ## calculate improvement over median baseline
    imprvt = round((rmse/baselines[i] - 1)*-100, 3)

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i]['silly_mean'] = [rmse, imprvt]

The root-mean-square error of english's mean model is 0.010981530956769198
The root-mean-square error of math's mean model is 0.019399287092353668
The root-mean-square error of rus_stackoverflow's mean model is 0.01763567702987531
The root-mean-square error of stackoverflow's mean model is 0.01411915241574049
The root-mean-square error of superuser's mean model is 0.010293770490765039


In [14]:
# improvement using rmse
pd.DataFrame.from_dict(RESULTS).T

,silly_mean,silly_median
english,"[0.010981530956769198, 3.732]","[0.011407298175493769, 0]"
math,"[0.019399287092353668, 0.464]","[0.01948979798344224, 0]"
rus_stackoverflow,"[0.01763567702987531, 6.797]","[0.018921842554517707, 0]"
stackoverflow,"[0.01411915241574049, 1.138]","[0.014281609584952339, 0]"
superuser,"[0.010293770490765039, 2.83]","[0.010593612389712697, 0]"


In [15]:
'''
Interesting that some models have a vast improvement just using a constant mean prediction compare to median.
'''

'\nInteresting that some models have a vast improvement just using a constant mean prediction compare to median.\n'

# Viewcount Model

In [16]:
from pyspark.ml.pipeline import Pipeline

from pyspark.ml.feature import VectorAssembler

## because you can't just pass a column without vectorising it
to_vectors = VectorAssembler(inputCols=["viewcount", "score"], outputCol="features")

processing_pipeline = Pipeline(stages=[to_vectors])

In [18]:
%%time

## linear regression on just viewcount
from pyspark.ml.regression import LinearRegression

'''## clear features column
for i in data_array:
    datasets[i] = datasets[i].drop("features").collect()'''
    
lr = LinearRegression(maxIter=100,
                      regParam=0.3,
                      elasticNetParam=0.8,
                      featuresCol="features",
                      labelCol="y_ravi",
                      predictionCol="viewcount_pred")

## make final pipeline
final_pipeline = Pipeline(stages=[to_vectors, lr])

## fit pipeline on datasets
for i in data_array:
    datasets[i] = final_pipeline.fit(datasets[i]).transform(datasets[i])

CPU times: user 195 ms, sys: 52.4 ms, total: 248 ms
Wall time: 43.9 s


In [19]:
## evaluate model
evaluator = RegressionEvaluator(metricName="rmse", labelCol="y_ravi", predictionCol="viewcount_pred")

for i in data_array:
    rmse = evaluator.evaluate(datasets[i])
    
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m viewcount\033[0m model is {rmse}")
    imprvt = round((rmse/baselines[i] - 1)*-100, 3)

    RESULTS[i]['viewcount'] = [rmse, imprvt]

The root-mean-square error of english's viewcount model is 0.010981530956769198
The root-mean-square error of math's viewcount model is 0.01939928709235388
The root-mean-square error of rus_stackoverflow's viewcount model is 0.017635677029875438
The root-mean-square error of stackoverflow's viewcount model is 0.014119152415740502
The root-mean-square error of superuser's viewcount model is 0.01029377049076506


In [20]:
# improvement using rmse
pd.DataFrame.from_dict(RESULTS).T

,silly_mean,silly_median,viewcount
english,"[0.010981530956769198, 3.732]","[0.011407298175493769, 0]","[0.010981530956769198, 3.732]"
math,"[0.019399287092353668, 0.464]","[0.01948979798344224, 0]","[0.01939928709235388, 0.464]"
rus_stackoverflow,"[0.01763567702987531, 6.797]","[0.018921842554517707, 0]","[0.017635677029875438, 6.797]"
stackoverflow,"[0.01411915241574049, 1.138]","[0.014281609584952339, 0]","[0.014119152415740502, 1.138]"
superuser,"[0.010293770490765039, 2.83]","[0.010593612389712697, 0]","[0.01029377049076506, 2.83]"


In [23]:
final_pipeline

Pipeline_b7354de8942a

In [21]:
show_spark_df(datasets['english'])

,body,title,viewcount,score,clean_body,y_ravi,median_pred,mean_pred,features,viewcount_pred
0,<p>How do I know when to use <em>lay</em> and ...,"What is the difference between ""lay"" and ""lie""?",18413,35,How do I know when to use lay and when to u...,0.001901,0.00069,0.003778,"[18413.0, 35.0]",0.003778
1,<p>When you want to connect two closely relate...,Should I use a semicolon or a dash to connect ...,106724,52,When you want to connect two closely related ...,0.000487,0.00069,0.003778,"[106724.0, 52.0]",0.003778
2,<blockquote>\n <p><strong>Possible Duplicate:...,"What does Maugham mean by ""his spaghetti were""?",1131,11,Possible Duplicate: Was the usage &...,0.009726,0.00069,0.003778,"[1131.0, 11.0]",0.003778
3,<p>How do you say it correctly?</p>\n\n<ul>\n<...,"""Adult and children stories"" or ""Adults and ch...",959,2,How do you say it correctly? Adult and c...,0.002086,0.00069,0.003778,"[959.0, 2.0]",0.003778
4,"<p>""Proven"" and ""proved"" both seem to mean the...","What is the difference between ""proven"" and ""p...",52711,50,"""Proven"" and ""proved"" both seem to mean the s...",0.000949,0.00069,0.003778,"[52711.0, 50.0]",0.003778


# Create Pipelines for Modeling

In [ ]:
## garbage collector to speed up computation
import gc
collected = gc.collect()
print("Garbage collector: collected %d objects." % collected)

In [ ]:
target = "y_ravi"
indep_text_variables = ["title", "clean_body"]
#.drop('age').collect()

In [ ]:
## import elements from natural language toolkit
import nltk
#nltk.download('all') # uncomment after first run as admin check
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop_words = set(stopwords.words('english'))
table = str.maketrans('', '', string.punctuation)
lmtzr = WordNetLemmatizer()

def get_tokens(line):
    '''
    Function to parse text features
    '''
    tokens = word_tokenize(line)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuations from each word
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stopwords
    words = [w for w in words if not w in stop_words]
    # lemmatizing the words, see https://en.wikipedia.org/wiki/Lemmatisation
    '''lemmatise or stem???'''
    words = [lmtzr.lemmatize(w) for w in words]
    # remove single letters
    words = [word for word in words if not len(word)==1]
    return (words)

In [ ]:
import nltk

from pyspark import keyword_only  ## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

## custom transformer for nltk tokenisation

class NLTKWordPunctTokenizer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, stopwords=None):
        super(NLTKWordPunctTokenizer, self).__init__()
        self.stopwords = Param(self, "stopwords", "")
        self._setDefault(stopwords=set())
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None, stopwords=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def setStopwords(self, value):
        self._paramMap[self.stopwords] = value
        return self

    def getStopwords(self):
        return self.getOrDefault(self.stopwords)

    def _transform(self, dataset):
        stopwords = self.getStopwords()

        def f(s):
            tokens = nltk.tokenize.wordpunct_tokenize(s)
            return [t for t in tokens if t.lower() not in stopwords]

        t = ArrayType(StringType())
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [ ]:
import pyspark.sql.functions as F
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import Bucketizer
from pyspark.sql import DataFrame
from typing import Iterable
import pandas as pd

## custom transformer to spread spares vectors into individual columns
class VectorMLliber(Transformer):
    """
    A custom Transformer which converts a column of pyspark.ml vectors to multiple pyspark.mllib vectors.
    """

    def __init__(self, inputCol=None):
        super(VectorMLliber, self).__init__()

    def _transform(self, df: DataFrame) -> DataFrame:
        
        def f(v):
            return Vectors.sparse(v.size, v.indices, v.values)
        
        df = df.rdd.map(lambda r: as_mllib_vector(r[0]))
        return df

In [ ]:
'''def as_mllib_vector(v):
    return Vectors.sparse(v.size, v.indices, v.values)

features = {}
feature_vec_list = {}
for i in data_array:
    features[i] = word_feat_list[i].select("features")
    feature_vec_list[i] = features[i].rdd.map(lambda r: as_mllib_vector(r[0]))
    feature_vec_list[i].cache()
'''

In [ ]:
nltk_tokenizer_body = NLTKWordPunctTokenizer(
    inputCol="clean_body", outputCol="body_words",  
    stopwords=set(nltk.corpus.stopwords.words('english')))

nltk_tokenizer_title = NLTKWordPunctTokenizer(
    inputCol="title", outputCol="title_words",  
    stopwords=set(nltk.corpus.stopwords.words('english')))

from pyspark.ml.feature import CountVectorizer, VectorAssembler

cnt_vectrizr_body = CountVectorizer(inputCol="body_words", outputCol="body_features", minDF=2)

cnt_vectrizr_title = CountVectorizer(inputCol="title_words", outputCol="title_features", minDF=2)

VectorMLliber_body = VectorMLliber(inputCol="body_features")

VectorMLliber_title = VectorMLliber(inputCol="body_title")



processing_pipeline = Pipeline(stages=[
    nltk_tokenizer_body, 
    nltk_tokenizer_title,
    cnt_vectrizr_body,
    cnt_vectrizr_title
])

In [ ]:
%%time

## check that pipeline is working (WHICH IT IS NOT)

data_processed = processing_pipeline.fit(datasets['english']).transform(datasets['english'])

show_spark_df(data_processed)

#data_processed.head(2)[0].features.values

In [ ]:
processing_ensembler = VectorAssembler(inputCols=["body_features", "title_features"], 
                                         outputCol="features")  

processing_pipeline = Pipeline(stages=[processing_pipeline, processing_ensembler])

In [ ]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(maxIter=100,
                      regParam=0.3,
                      elasticNetParam=0.8,
                      featuresCol="features",
                      labelCol="y_ravi",
                      predictionCol="prediction")

# fit linear regression pipeline
pipeline = Pipeline(stages=[processing_pipeline, lr])
trained_pipeline = pipeline.fit(datasets['english'])
trained_pipeline

In [ ]:
show_spark_df(trained_pipeline.transform(datasets['english']))

In [ ]:
(trained_pipeline
 .transform(datasets['english'])
 .select(
    indep_text_variables + ["prediction"]
 )
 .write
 .parquet("linreg_prediction.parquet")
)

In [ ]:
linreg_predictions = spark.read.parquet("linreg_prediction.parquet")

In [ ]:
linreg_predictions.toPandas().head()

In [ ]:
linreg_predictions.select("prediction").describe().toPandas()

# Save pipelines

In [ ]:
from joblib import dump, load
dump(estimator_pipeline, 'pipeline.joblib') 

reloaded = load("pipeline.joblib")

Now we can predict directly!

reloaded.predict(X)[:10]

# Convert notebook to python file

In [ ]:
!jupyter nbconvert --to script 0-master-notebook-pipelines.ipynb